In [ ]:
%load_ext autoreload
%autoreload 2
import logging
logging.basicConfig(level=logging.INFO)
import matplotlib.pyplot as plt

import geopandas as gpd
from eodag import EODataAccessGateway
import rioxarray as riox
import numpy as np
import folium
from folium import LayerControl
from eo_tools_dev.util import show_cog

# credentials need to be stored in the following file (see EODAG docs)
confpath = "/data/eodag_config.yml"
dag = EODataAccessGateway(user_conf_file_path=confpath)
# make sure cop_dataspace will be used 
dag.set_preferred_provider("cop_dataspace")
log = logging.getLogger(__name__)

## Set up parameters and output dir

In [ ]:
# change to your custom locations
data_dir = "/data/S1"

# Morocco products
# ids = [
#  "S1A_IW_SLC__1SDV_20230904T063730_20230904T063757_050174_0609E3_DAA1", 
#  "S1A_IW_SLC__1SDV_20230916T063730_20230916T063757_050349_060FCD_6814"
# ]
# Berlin products
ids = [
 "S1A_IW_SLC__1SDV_20240916T165234_20240916T165301_055693_06CD1B_C701",
 "S1A_IW_SLC__1SDV_20240928T165234_20240928T165301_055868_06D40C_581D", 
]
primary_dir = f"{data_dir}/{ids[0]}.zip"
secondary_dir = f"{data_dir}/{ids[1]}.zip"
outputs_prefix="/data/res/test-new-coherence-berlin"

## Download S-1 products

In [ ]:
# load a geometry
# file_aoi = "../data/Morocco_AOI.geojson"
file_aoi = "../data/Berlin.geojson"
shp = gpd.read_file(file_aoi).geometry[0]

# search_criteria = {
#     "productType": "S1_SAR_SLC",
#     "start": "2023-09-03",
#     "end": "2023-09-17",
#     "geom": shp
# }

# results, _ = dag.search(**search_criteria)
# to_dl = [it for it in results if it.properties["id"] in ids]
# print(f"{len(to_dl)} products to download")
# dag.download_all(to_dl, outputs_prefix="/data/S1/", extract=False)

## Pre-process InSAR pair

In [ ]:
from eo_tools.S1.process import prepare_insar

out_dir = prepare_insar(
    dir_prm=primary_dir,
    dir_sec=secondary_dir,
    outputs_prefix=outputs_prefix,
    aoi_name=None,
    shp=shp,
    pol="vh",
    subswaths=["IW1", "IW2", "IW3"],
    cal_type="beta",
    apply_fast_esd=False,
    dem_upsampling=1.8,
    dem_force_download=False,
    dem_buffer_arc_sec=40,
    warp_kernel="bicubic",
)

## Apply processing chains: coherence and change detection

In [ ]:
from eo_tools.S1.process import coherence, multilook
from eo_tools.S1.process import apply_to_patterns_for_pair, apply_to_patterns_for_single
from pathlib import Path

out_dir = list(Path(outputs_prefix).glob("S1*/sar"))[0]

geo_dir = Path(out_dir).parent

# compute interferometric coherence
apply_to_patterns_for_pair(
    coherence,
    out_dir=out_dir,
    file_prm_prefix="slc_prm",
    file_sec_prefix="slc_sec",
    file_out_prefix="coh_new",
    box_size=[3, 3],
    multilook=[1, 4],
)


apply_to_patterns_for_pair(
    coherence,
    out_dir=out_dir,
    file_prm_prefix="slc_prm",
    file_sec_prefix="slc_sec",
    file_out_prefix="coh_old",
    box_size=[3, 11],
    multilook=[1, 1],
)

apply_to_patterns_for_single(
    multilook,
    out_dir=out_dir,
    file_in_prefix="coh_old",
    file_out_prefix="coh_old",
    mlt=[1, 4],
)

## Apply geocoding, merge and crop subswaths

In [ ]:
from eo_tools.S1.process import geocode_and_merge_iw
geo_dir = Path(out_dir).parent
geocode_and_merge_iw(geo_dir, shp=shp, var_names=["coh_new","coh_old"])

## Visualize
Click on top right icon to hide/show layers

In [ ]:
m = folium.Map()
# _ = show_cog(f"{geo_dir}/coh_old_vv.tif", m, rescale="0,1")
_ = show_cog(f"{geo_dir}/coh_old_vh.tif", m, rescale="0,1")
# _ = show_cog(f"{geo_dir}/coh_new_vv.tif", m, rescale="0,1")
_ = show_cog(f"{geo_dir}/coh_new_vh.tif", m, rescale="0,1")
LayerControl().add_to(m)
m